In [ ]:
pip install spade

# Ejemplo de Contract Net Protocol

**Recuerda cambiar el nombre de los agentes**

# Manager Agent: inicia la solicitud de propuestas
Genera un numero aleatorio y seleccionará la propuesta de aquel agente que propoga el valor que mas se acerque

In [ ]:
import spade
import random
import asyncio
from spade.message import Message
from spade.behaviour import CyclicBehaviour, OneShotBehaviour

class ManagerAgent(spade.agent.Agent):
    def __init__(self, jid, password, num_contractors):
        super().__init__(jid, password)
        self.contractors = []  # Lista dinámica de contratistas
        self.max_contractors = num_contractors
    
    async def setup(self):
        self.add_behaviour(self.RegisterBehaviour())

    
    class RegisterBehaviour(CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg and msg.metadata["performative"] == "register":
                contractor_jid = str(msg.sender)
                if contractor_jid not in self.agent.contractors:
                    self.agent.contractors.append(contractor_jid)
                    print(f"Manager: Agente {contractor_jid} registrado.")
           
            if len(self.agent.contractors) == self.agent.max_contractors: 
                self.agent.add_behaviour(self.agent.CFPBehaviour())        
                self.kill()
            

    class CFPBehaviour(CyclicBehaviour):
        async def run(self):

            if len(self.agent.contractors)< self.agent.max_contractors: return
            
            target_value = random.randint(1, 100)
            print(f"Manager: El valor objetivo es {target_value}")


            # Enviar CFP a todos los agentes contratistas
            cfp = Message()
            cfp.set_metadata("performative", "call_for_proposals")
            cfp.body = str(target_value)
            for agent_jid in self.agent.contractors:
                cfp.to = agent_jid
                await self.send(cfp)

                
            await asyncio.sleep(5)
            
            # Recibir propuestas y seleccionar la más cercana
            closest_value = None
            closest_agent = None
            for _ in range(len(self.agent.contractors)):
                msg = await self.receive(timeout=10)
                if msg and msg.metadata.get("performative") == "proposal":
                    proposal_value = int(msg.body)
                    if closest_value is None or abs(proposal_value - target_value) < abs(closest_value - target_value):
                        closest_value = proposal_value
                        closest_agent = msg.sender

            print(f"Manager: Agente {closest_agent} seleccionado con la propuesta {closest_value}")

            # Enviar mensajes de aceptación y rechazo
            for agent_jid in self.agent.contractors:
                response_msg = Message(to=agent_jid)
                if agent_jid == str(closest_agent):
                    response_msg.set_metadata("performative", "accept_proposal")
                else:
                    response_msg.set_metadata("performative", "reject_proposal")
                await self.send(response_msg)

            self.kill()

# Contractor Agent: responde con una propuesta
Responde a una propuesta con un valor aleatorio

**Recuerda cambiar el nombre de los agentes**

In [ ]:
class ContractorAgent(spade.agent.Agent):
    async def setup(self):
        self.add_behaviour(self.RegisterBehaviour())


    class RegisterBehaviour(CyclicBehaviour):
        async def run(self):
            # Enviar mensaje de registro al manager
            register_msg = Message(to="manager_vjulian@gtirouter.dsic.upv.es")
            register_msg.set_metadata("performative", "register")
            await self.send(register_msg)
            self.agent.add_behaviour(self.agent.ProposalBehaviour())
            self.agent.add_behaviour(self.agent.HandleResponseBehaviour())
            self.kill()
            
    class ProposalBehaviour(OneShotBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg: #and msg.metadata["performative"] == "call_for_proposals":
                proposal_value = random.randint(1, 100)
                print(f"{self.agent.jid} propone {proposal_value}")

                reply = msg.make_reply()
                reply.set_metadata("performative", "proposal")
                reply.body = str(proposal_value)
                await self.send(reply)

    class HandleResponseBehaviour(CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg:
                if msg.metadata["performative"] == "accept_proposal":
                    print(f"{self.agent.jid} ha sido aceptado para la tarea.")
                elif msg.metadata["performative"] == "reject_proposal":
                    print(f"{self.agent.jid} ha sido rechazado para la tarea.")


In [ ]:

num_contractors = 5

manager = ManagerAgent("manager_vjulian@gtirouter.dsic.upv.es", "secret", num_contractors)
await manager.start()


contractors = [ContractorAgent(f"contratista{i}vjulian@gtirouter.dsic.upv.es", "secret") for i in range(1, num_contractors+1)]
for contractor in contractors:
    await contractor.start()



await asyncio.sleep(5)  # Tiempo para permitir la interacción

await manager.stop()
for contractor in contractors:
    await contractor.stop()


